Capture system table descriptions and sample data. 
Save output and feed to LLM to generate SQL.

In [0]:
# List of system catalog schemas
schemas = ["access", "ai", "billing", "compute", "query", "serving", "storage", "lakeflow", "information_schema"]

# Function to describe a table and capture 10 sample rows
def describe_and_sample(schema, table):
    # Describe the table
    description = spark.sql(f"DESCRIBE system.{schema}.{table}").collect()
    
    # Capture 10 sample rows
    sample_rows = spark.sql(f"SELECT * FROM system.{schema}.{table} LIMIT 10").collect()
    
    return description, sample_rows

# Open the file to write the output
with open("/Volumes/juan_dev/data_eng/data/output.txt", "w") as file:
    # Iterate over each schema and table
    for schema in schemas:
        # Get the list of tables in the schema
        tables = spark.sql(f"SHOW TABLES IN system.{schema}").select("tableName").rdd.flatMap(lambda x: x).collect()
        
        for table in tables:
            description, sample_rows = describe_and_sample(schema, table)
            
            # Write the description and sample rows to the file
            file.write(f"Description of {schema}.{table}:\n")
            for row in description:
                file.write(f"{row}\n")
            
            file.write(f"\nSample rows from {schema}.{table}:\n")
            for row in sample_rows:
                file.write(f"{row}\n")
            file.write("\n" + "="*50 + "\n")